# Constructing a Grid from Points

In [ ]:
import uxarray as ux
import xarray as xr
import cartopy.crs as ccrs
import geoviews as gv

## Data

In [ ]:
xrds = xr.open_dataset("../../test/meshfiles/points/outCSne30/psi-points.nc")
xrds

In [ ]:
points = (xrds.longitude.values, xrds.latitude.values)

In [ ]:
gv_points = gv.Points(points, crs=ccrs.PlateCarree()).opts(
    projection=ccrs.Robinson(), width=700, height=350, color="red"
)
gv_points

## Spherical Voronoi

In [ ]:
grid_sv = ux.Grid.from_points(points, method="spherical_voronoi")

In [ ]:
grid_sv.plot(projection=ccrs.Robinson(), periodic_elements="split") * gv_points